In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/videogamesales/vgsales.csv


# Importing libraries and Data

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly
import matplotlib.pyplot as plt
import re

from plotly.subplots import make_subplots
from scipy.stats import chi2_contingency
from wordcloud import wordcloud

In [3]:
df_train = pd.read_csv('../input/videogamesales/vgsales.csv')

df_train.head(5)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [4]:
df_train.info()

## as seen below 2 columns are missing values. 
## we will drop the samples with missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [5]:
##1. Drop the samples with missing values
##2. index needs to be reset after deleted samples
##3. updating rank column after reset

df_train = df_train.dropna()

df_train = df_train.reset_index(drop = True)

df_train.loc[:, 'Rank'] = np.arange(df_train.shape[0])+1

As shown in df_train.info...the year is showing float. We will change that to an int for display

In [6]:
df_train['Year'] = df_train['Year'].astype(int)

# Analysis and Visualization

In [7]:
fig = px.box(df_train, y="Global_Sales", points="all", height = 400, color_discrete_sequence = px.colors.sequential.Plasma_r)


fig.show()

As shown in the figure above, most of the sales fall within the value range of 10. We will use that information with the next plots.

In [8]:
t1 = df_train[df_train['Global_Sales'] < 10]

fig = px.histogram(t1, x ="Global_Sales", height = 400, color_discrete_sequence = ['#A4CCD9'])

fig.show()

In [9]:
t2 = df_train[df_train["Global_Sales"] > 10]

fig = px.histogram(t2, x = "Global_Sales", height = 400, color_discrete_sequence = ["powderblue"])


fig.show()

By combining the two previous figures above, it can be found that the number of games with a total sales value of more than 10 is relatively small, and the sales value of most games is in the range of 0-1, which to a certain extent reflects the two-eight law of sales in the game industry.

Changes in sales over time

In [10]:
## Total volume of sales

df_yearcount = df_train.groupby(df_train['Year'])[['Rank']].count().rename(columns={'Rank':'counts'})
df_yearsales = df_train.groupby(df_train['Year'])[['Global_Sales']].sum()

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Bar(x=df_yearcount.index, y=df_yearcount['counts'], marker=dict(color='rgba(17, 145, 171, 0.6)'), name = 'counts'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=df_yearsales.index, y=df_yearsales['Global_Sales'], name='Global_Sales'),
    secondary_y=True,
)
fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="<b>counts</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Global_Sales</b>", secondary_y=True)
fig.show()

As you can see from the figure, after a small dip in sales 2003/2004,sales exploded and peaked 2008-2010. Also known as the golden age of gaming. The sales volume in a year is often proportional of the games released in the given year. However, one outlier being 2004, where in my opinion, the resale/collectivity of games began to gain traction.



Lets take a deeper dive into the individual gaming systems/games, and the platforms per game sales

In [11]:
##overall observations of the video game market
df_platcount = df_train.groupby(df_train['Platform'])[['Rank']].count().rename(columns = {'Rank':'counts'}).sort_values('counts', ascending = False)

fig = px.bar(df_platcount, x=df_platcount.index, y='counts', color='counts',color_continuous_scale=['rgba(17, 171, 122, 0.6)', 'rgba(17, 145, 171, 0.6)'],
              height=400)
fig.show()

Platforms representing the top 100 games sold

In [12]:
## this observation is representative of the platforms responsible for the top 100 games

df_platcount100 = df_train[0:100].groupby(df_train['Platform'])[['Rank']].count().rename(columns = {'Rank':'counts'}).sort_values('counts', ascending = False)


fig = px.bar(df_platcount100, x = df_platcount100.index, y = 'counts', color = 'counts', color_continuous_scale = ['rgba(17, 171, 122. 0.6)', 'rgba(17, 145, 171, 0.6)'], height = 400)

fig.show()

Here you can see the major outlier when it comes to top 100 games sold. With Xbox360, Wii, and Nintendo DS pulling most of the weight of the top 100 list. By now, we see that not only do Xbox360 and Nintendo Wii have a high volume of games(although not higher than some), they do make quality games based on how many games they have that are in the top 100 games sold.

Top 10 games by platform

In [13]:
df_platcount10 = df_train[0:10].groupby(df_train['Platform'])[['Rank']].count().rename(columns = {'Rank':'counts'}).sort_values('counts', ascending = False)

fig = px.bar(df_platcount10, x = df_platcount10.index, y ='counts', color ='counts', color_continuous_scale = ['rgba(17, 171, 122, 0.6)', 'rgba(17, 145, 171, 0.6)'],
              height=400)
fig.show()

In [14]:
top5_plat = ['Nintendo', 'Sony Computer Entertainment','Microsoft Game Studios','Konami Digital Entertainment','Electronic Arts']

perc = df_train.loc[:,["Year","Publisher",'Global_Sales']]
perc['total_sales'] = perc.groupby([perc.Publisher,perc.Year])['Global_Sales'].transform('sum')
perc.drop('Global_Sales', axis=1, inplace=True)
perc = perc.drop_duplicates()
perc = perc[(perc['Year']>=2006)]
perc = perc.sort_values("Year",ascending = False)
perc = perc.loc[perc['Publisher'].isin(top5_plat)]
perc = perc.sort_values("Year")

fig=px.bar(perc,x='Publisher', y="total_sales", animation_frame="Year", 
           animation_group="Publisher", color="Publisher", hover_name="Publisher",range_y=[0,200])

fig.update_layout(title_text="Top 5 Platforms Game Sale by Year", xaxis_domain=[0.05, 1.0])


fig.show()